In [1]:
import pandas as pd
import numpy as np

# 1. Load in datasets

In [2]:
addr_train = "C:/Users/poduo/Anaconda3_JupyterNB/SC00030 Python ML & DL/HW_Practice/Kaggle_Titanic/DataSet/train.csv"
addr_test = "C:/Users/poduo/Anaconda3_JupyterNB/SC00030 Python ML & DL/HW_Practice/Kaggle_Titanic/DataSet/test.csv"

train = pd.read_csv(addr_train)
test = pd.read_csv(addr_test)

ori_train = train.copy()
ori_test = test.copy()

full_data = [train, test]

In [3]:
train.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
370,371,1,1,"Harder, Mr. George Achilles",male,25.0,1,0,11765,55.4417,E50,C
580,581,1,2,"Christy, Miss. Julie Rachel",female,25.0,1,1,237789,30.0000,NaN,S
867,868,0,1,"Roebling, Mr. Washington Augustus II",male,31.0,0,0,PC 17590,50.4958,A24,S
693,694,0,3,"Saad, Mr. Khalil",male,25.0,0,0,2672,7.2250,NaN,C
470,471,0,3,"Keefe, Mr. Arthur",male,NaN,0,0,323592,7.2500,NaN,S


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


# 2. Feature Engineering
form https://www.kaggle.com/sinakhorami/titanic-best-working-classifier

### Pclass 艙等
名義量尺

1 = 1st, 2 = 2nd, 3 = 3rd

In [6]:
print (train[['Pclass', 'Survived']].groupby(['Pclass'], as_index = False).mean())

   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363


### Sex 性別
名義量尺

In [7]:
print (train[['Sex', 'Survived']].groupby(['Sex'], as_index= False).mean())

      Sex  Survived
0  female  0.742038
1    male  0.188908


### SibSp & Parch
比率量尺

In [8]:
# 計算 家庭成員 + 自己
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp']+ dataset['Parch']+ 1

print(train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index= False).mean())

   FamilySize  Survived
0           1  0.303538
1           2  0.552795
2           3  0.578431
3           4  0.724138
4           5  0.200000
5           6  0.136364
6           7  0.333333
7           8  0.000000
8          11  0.000000


### 新增 IsAlone 欄位
1 獨自乘船；0 與家人乘船

名義量尺

In [9]:
#非獨自一人 0(False，有家庭成員)；獨自一人 1(True，沒有家庭成員)
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
print (train[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index= False).mean())

   IsAlone  Survived
0        0  0.505650
1        1  0.303538


### Embarked 
上船地點：名義量尺

C = Cherbourg, Q = Queenstown, S = Southampton

In [10]:
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
    
print (train[['Embarked', 'Survived']].groupby(['Embarked'], as_index= False).mean())

  Embarked  Survived
0        C  0.553571
1        Q  0.389610
2        S  0.339009


### Fare
票價：比率量尺
#### 遺失值：補入中位數

In [11]:
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())

# 使用 4 分位數 做分箱
train['CategoricalFare'] = pd.qcut(train['Fare'],4)
print (train[['CategoricalFare', 'Survived']].groupby(['CategoricalFare'], as_index= False).mean())

   CategoricalFare  Survived
0   (-0.001, 7.91]  0.197309
1   (7.91, 14.454]  0.303571
2   (14.454, 31.0]  0.454955
3  (31.0, 512.329]  0.581081


### Age
年齡；比率量尺
#### 遺失值：隨機填入 平均+-標準差 的值
作 分箱pcut

In [12]:
# 將 遺失值 隨機放入 平均值+-標準差範圍的 整數
for dataset in full_data:
    age_mean = train['Age'].mean() # dataset['Age'].mean()
    age_std = train['Age'].std() # dataset['Age'].std()
    
    age_null_count = dataset['Age'].isnull().sum()
    age_random_list = np.random.randint(age_mean - age_std, age_mean + age_std, size = age_null_count)
    #     series = series
    dataset['Age'][np.isnan(dataset['Age'])] = age_random_list
    dataset['Age'] = dataset['Age'].astype(int)

    # 分箱 right = True ， <=
train['CategoricalAge'] = pd.cut(train['Age'], 5)
print (train[['CategoricalAge', 'Survived']].groupby(['CategoricalAge'], as_index= False).mean())

  CategoricalAge  Survived
0  (-0.08, 16.0]  0.535714
1   (16.0, 32.0]  0.353982
2   (32.0, 48.0]  0.368421
3   (48.0, 64.0]  0.434783
4   (64.0, 80.0]  0.090909


C:\Users\poduo\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


### Name

In [13]:
import re

In [14]:
def get_title(name):
    title = re.search(' ([\w]+)\.', name)
    if title:
        return title.group(1)
    return ''
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)

In [15]:
print(pd.crosstab(train['Title'], train['Sex']))

Sex       female  male
Title                 
Capt           0     1
Col            0     2
Countess       1     0
Don            0     1
Dr             1     6
Jonkheer       0     1
Lady           1     0
Major          0     2
Master         0    40
Miss         182     0
Mlle           2     0
Mme            1     0
Mr             0   517
Mrs          125     0
Ms             1     0
Rev            0     6
Sir            0     1


In [16]:
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
                                                 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace(['Mlle', 'Ms'], 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

print (train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean())

    Title  Survived
0  Master  0.575000
1    Miss  0.702703
2      Mr  0.156673
3     Mrs  0.793651
4    Rare  0.347826


# MY Data Clean

## One-hot-Encoding

In [17]:
# 如果這樣寫，沒辦法存回 train 和 test
for dataset in full_data:
    dum_sex_df = pd.get_dummies(dataset['Sex'])
    dataset = dataset.join(dum_sex_df)

In [18]:
full_data[0]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone,CategoricalFare,CategoricalAge,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,2,0,"(-0.001, 7.91]","(16.0, 32.0]",Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,2,0,"(31.0, 512.329]","(32.0, 48.0]",Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,1,1,"(7.91, 14.454]","(16.0, 32.0]",Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,2,0,"(31.0, 512.329]","(32.0, 48.0]",Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S,1,1,"(7.91, 14.454]","(32.0, 48.0]",Mr
5,6,0,3,"Moran, Mr. James",male,43,0,0,330877,8.4583,NaN,Q,1,1,"(7.91, 14.454]","(32.0, 48.0]",Mr
6,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S,1,1,"(31.0, 512.329]","(48.0, 64.0]",Mr
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.0750,NaN,S,5,0,"(14.454, 31.0]","(-0.08, 16.0]",Master
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,NaN,S,3,0,"(7.91, 14.454]","(16.0, 32.0]",Mrs
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,NaN,C,2,0,"(14.454, 31.0]","(-0.08, 16.0]",Mrs


### Sex

In [17]:
dum_sex_df = pd.get_dummies(train['Sex'])
train = train.join(dum_sex_df)

In [18]:
dum_sex_df = pd.get_dummies(test['Sex'])
test = test.join(dum_sex_df)

### Title

In [19]:
train['Title'] = train['Title'].fillna(0)
dum_df = pd.get_dummies(train['Title'])
train = train.join(dum_df)

In [20]:
test['Title'] = test['Title'].fillna(0)
dum_df = pd.get_dummies(test['Title'])
test = test.join(dum_df)

### Embarked

In [21]:
dum_df = pd.get_dummies(train['Embarked'])
train = train.join(dum_df)

In [22]:
dum_df = pd.get_dummies(test['Embarked'])
test = test.join(dum_df)

In [23]:
full_data2 = [train, test]

In [24]:
for dataset in full_data2:
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare'] = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4

In [25]:
# Feature Selection
drop_elements = ['PassengerId', 'Name', 'Sex', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked', 'FamilySize', 'Title']

train = train.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)

test = test.drop(drop_elements, axis = 1)

print (train.head(5))
print (test.head(5))

train_values = train.values
test_values = test.values

   Survived  Pclass  Age  Fare  IsAlone  female  male  Master  Miss  Mr  Mrs  \
0         0       3    1     0        0       0     1       0     0   1    0   
1         1       1    2     3        0       1     0       0     0   0    1   
2         1       3    1     1        1       1     0       0     1   0    0   
3         1       1    2     3        0       1     0       0     0   0    1   
4         0       3    2     1        1       0     1       0     0   1    0   

   Rare  C  Q  S  
0     0  0  0  1  
1     0  1  0  0  
2     0  0  0  1  
3     0  0  0  1  
4     0  0  0  1  
   Pclass  Age  Fare  IsAlone  female  male  Master  Miss  Mr  Mrs  Rare  C  \
0       3    2     0        1       0     1       0     0   1    0     0  0   
1       3    2     0        0       1     0       0     0   0    1     0  0   
2       2    3     1        1       0     1       0     0   1    0     0  0   
3       3    1     1        1       0     1       0     0   1    0     0  0   
4       3 

In [26]:
train_values

array([[0, 3, 1, ..., 0, 0, 1],
       [1, 1, 2, ..., 1, 0, 0],
       [1, 3, 1, ..., 0, 0, 1],
       ..., 
       [0, 3, 1, ..., 0, 0, 1],
       [1, 1, 1, ..., 1, 0, 0],
       [0, 3, 1, ..., 0, 1, 0]], dtype=int64)

# ML

In [33]:
from sklearn.model_selection import train_test_split
X = train.drop('Survived', axis= 1)
y = train[['Survived']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)

Pclass 是否要 Label encoding ?